In [607]:
# Config Data Structure
import pandas as pd
from datetime import datetime as dt
from pymongo import MongoClient as mc
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter
import pprint as pp
import random as ran
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM

mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)
keti_db = client.keti_pattern_recognition

household_col = keti_db.household_info
weather_col = keti_db.weather_info

In [608]:
# TimeSlot In
hh_db_datas = household_col.find_one({"uid": "아파트1-104-1206"})
hh_db_datas

uid_in, timeslot = hh_db_datas['uid'], hh_db_datas['timeslot']

datelist = [
    dt.strptime(ts['time'], "%Y-%m-%d T%H:%M %z").date()
    for ts in timeslot
]
datelist = list(set(datelist))
datelist.sort()

ts_datas = {}
start_idx = 0
end_idx = 96
enl = 1

for date in datelist:
    ts_datas[date] = [ts['power'] *
                      enl for ts in timeslot[start_idx:end_idx]]
    start_idx = end_idx
    end_idx = end_idx + 96

ts_datas = pd.DataFrame(ts_datas).T
hh_datas = ts_datas.reset_index().copy()

hh_datas.rename(columns={"index": "date"}, inplace=True)
hh_datas['date'] = pd.to_datetime(hh_datas['date'])

hh_datas.set_index('date', inplace=True)

# Merging
merge_size = 4
merge_datas = pd.DataFrame()
for date in hh_datas.index:
    merge_ts = []
    new_ts_size = round(len(hh_datas.loc[date]) / merge_size)
    
    for idx in range(0,new_ts_size):
        merge_ts.append(
            round(hh_datas.loc[date][merge_size * idx:merge_size * (idx + 1)].sum(), 3)
        )
    merge_datas[date] = merge_ts
    
def get_season_no(month):
    if month in [3,4,5]:
        return 1 # 봄
    elif month in [6,7,8]:
        return 2 # 여름
    elif month in [9,10,11]:
        return 3 # 가을
    elif month in [12,1,2]:
        return 4 # 겨울
    
separate_datas_col = list(filter(lambda data: get_season_no(data.month) == 1, merge_datas.columns))
# print(separate_datas_col)
merge_datas = merge_datas[separate_datas_col]
merge_datas

,2018-05-01,2018-05-02,2018-05-03,2018-05-04,2018-05-05,2018-05-06,2018-05-07,2018-05-08,2018-05-09,2018-05-10,...,2019-04-21,2019-04-22,2019-04-23,2019-04-24,2019-04-25,2019-04-26,2019-04-27,2019-04-28,2019-04-29,2019-04-30
0,0.341,0.275,0.183,0.309,0.305,0.397,0.347,0.345,0.312,0.321,...,0.182,0.060,0.063,0.066,0.149,0.164,0.130,0.290,0.056,0.045
1,0.337,0.201,0.235,0.308,0.179,0.409,0.178,0.272,0.191,0.208,...,0.209,0.038,0.049,0.062,0.052,0.063,0.046,0.267,0.053,0.044
2,0.324,0.176,0.167,0.309,0.180,0.400,0.173,0.206,0.183,0.203,...,0.197,0.050,0.041,0.046,0.067,0.065,0.042,0.244,0.062,0.059
3,0.319,0.210,0.165,0.309,0.172,0.384,0.176,0.204,0.173,0.189,...,0.194,0.060,0.064,0.051,0.054,0.039,0.060,0.276,0.063,0.058
4,0.235,0.199,0.163,0.311,0.171,0.276,0.178,0.173,0.170,0.184,...,0.046,0.054,0.061,0.064,0.041,0.056,0.033,0.232,0.061,0.033
5,0.169,0.202,0.164,0.217,0.174,0.206,0.182,0.180,0.170,0.172,...,0.066,0.035,0.035,0.064,0.066,0.063,0.054,0.135,0.087,0.054
6,0.200,0.199,0.162,0.218,0.169,0.200,0.180,0.188,0.207,0.169,...,0.068,0.062,0.057,0.047,0.063,0.071,0.059,0.093,0.050,0.059
7,0.171,0.252,0.173,0.212,0.164,0.199,0.232,0.179,0.215,0.216,...,0.049,0.060,0.063,0.049,0.039,0.046,0.028,0.093,0.046,0.049
8,0.170,0.213,0.226,0.242,0.162,0.199,0.221,0.212,0.244,0.169,...,0.050,0.040,0.046,0.064,0.059,0.062,0.059,0.066,0.051,0.040
9,0.172,0.173,0.178,0.187,0.166,0.200,0.217,0.171,0.204,0.170,...,0.141,0.050,0.046,0.063,0.065,0.049,0.047,0.089,0.060,0.058


In [609]:
# Set Test Datas
# 계절별 10% 랜덤하게
def get_season_no(month):
    if month in [3,4,5]:
        return 1 # 봄
    elif month in [6,7,8]:
        return 2 # 여름
    elif month in [9,10,11]:
        return 3 # 가을
    elif month in [12,1,2]:
        return 4 # 겨울
    
test_merge_datas = pd.DataFrame();

for i in range(1,2):
    filter_list = list(filter(lambda date: get_season_no(date.month) == i, merge_datas.columns))
    test_list_idx = list()
    while True:
        filter_data = filter_list[ran.randrange(0,len(filter_list))]
        if filter_data not in test_list_idx:
            test_list_idx.append(filter_data)
            
        if len(test_list_idx) >= (len(filter_list) * 15 / 100):
            break;
    test_merge_datas = pd.concat([test_merge_datas, merge_datas[test_list_idx]], axis=1)
    merge_datas.drop(test_list_idx, axis=1, inplace=True)

test_merge_datas

,2019-04-29,2018-05-02,2018-05-24,2019-03-14,2019-03-30,2018-05-11,2019-03-10,2018-05-23,2019-04-13,2019-04-27,2018-05-08,2019-03-18,2019-03-28,2019-03-05
0,0.056,0.275,0.176,0.034,0.277,0.182,0.304,0.257,0.227,0.130,0.345,0.140,0.092,0.113
1,0.053,0.201,0.175,0.051,0.128,0.175,0.285,0.181,0.094,0.046,0.272,0.137,0.085,0.109
2,0.062,0.176,0.211,0.101,0.078,0.172,0.296,0.174,0.073,0.042,0.206,0.103,0.070,0.089
3,0.063,0.210,0.211,0.127,0.097,0.168,0.300,0.171,0.081,0.060,0.204,0.094,0.092,0.113
4,0.061,0.199,0.204,0.130,0.089,0.167,0.285,0.174,0.094,0.033,0.173,0.060,0.090,0.113
5,0.087,0.202,0.202,0.140,0.069,0.166,0.193,0.171,0.093,0.054,0.180,0.084,0.065,0.105
6,0.050,0.199,0.176,0.055,0.180,0.167,0.131,0.172,0.071,0.059,0.188,0.127,0.093,0.094
7,0.046,0.252,0.241,0.097,0.152,0.228,0.130,0.217,0.083,0.028,0.179,0.066,0.095,0.123
8,0.051,0.213,0.175,0.072,0.039,0.186,0.155,0.188,0.094,0.059,0.212,0.064,0.066,0.111
9,0.060,0.173,0.174,0.092,0.063,0.223,0.103,0.189,0.088,0.047,0.171,0.058,0.091,0.096


In [610]:
from functools import reduce

merge_datas
y = reduce(lambda acc, cur: cur + acc ,merge_datas.values.tolist(), [])
value_size = len(y)
print(value_size)

1872


In [611]:
# Config Training Datas
wt_db_datas = weather_col.find()
wt_datas = pd.DataFrame()
for wt in wt_db_datas:
    tmp = pd.DataFrame()
    tmp['date'] = [wt['date']]
    tmp['weather'] = [wt['weather']]
    tmp['avg_ta'] = round(float(wt['avgTa']))
    tmp['avg_rhm'] = round(float(wt['avgRhm']))
    
    wt_datas = pd.concat([wt_datas, tmp])

# weather 정수 인코딩
weather_count = Counter(wt_datas['weather'])
weather_integer = dict()
rank = 1
for key, count in weather_count.most_common():
    weather_integer[key] = rank
    rank += 1
pp.pprint(weather_integer)
wt_datas['weather_no'] = [weather_integer[weather] for weather in wt_datas['weather']]
wt_datas

def get_season_no(month):
    if month in [3,4,5]:
        return 1 # 봄
    elif month in [6,7,8]:
        return 2 # 여름
    elif month in [9,10,11]:
        return 3 # 가을
    elif month in [12,1,2]:
        return 4 # 겨울
    
# Date, Season Utils
wt_datas['season_no'] = [get_season_no(weather.month) for weather in wt_datas['date']] 
wt_datas['day_no'] = [weather.weekday() + 1 for weather in wt_datas['date']] 

sample_weather_col = ['season_no','day_no','weather_no','avg_ta', 'avg_rhm']
sample_weather_col_2 = ['date','season_no','day_no','weather','weather_no','avg_ta', 'avg_rhm']
wt_datas[sample_weather_col_2]

{'눈': 5,
 '박무': 4,
 '비': 3,
 '소나기': 7,
 '안개': 10,
 '안개비': 11,
 '연무': 2,
 '진눈깨비': 12,
 '채운': 8,
 '특이사항 없음': 1,
 '햇무리': 6,
 '황사': 9}


,date,season_no,day_no,weather,weather_no,avg_ta,avg_rhm
0,2018-05-01,1,2,연무,2,20,73
0,2018-05-02,1,3,비,3,15,90
0,2018-05-03,1,4,비,3,11,62
0,2018-05-04,1,5,특이사항 없음,1,14,46
0,2018-05-05,1,6,특이사항 없음,1,18,47
...,...,...,...,...,...,...,...
0,2019-04-26,1,5,비,3,8,82
0,2019-04-27,1,6,햇무리,6,12,53
0,2019-04-28,1,7,비,3,13,49
0,2019-04-29,1,1,햇무리,6,13,53


In [612]:
# Config Sample Datas - Padding
# Data Preprocessing
def get_samples(datas):
    samples = list()
    for col in datas:
        timeslot = datas[col].values.tolist()
        weather = wt_datas[wt_datas['date'] == col][sample_weather_col].values.tolist()[0]
        for time in range(1,25):
            samples.append(weather + timeslot[:time])

    print("Samples Before Padding Process")
    print(np.array(samples))
    print("\nTranining Sample Size : {}".format(len(samples)))
    
    return samples

def get_padding_samples(samples):    
    # Padding
    pad_samples = list()
    SAMPLE_MAX_LEN = max([len(s) for s in samples])
    print("Tranining Sample MAX_LEN : {}".format(SAMPLE_MAX_LEN))
    for sample in samples:
        err = SAMPLE_MAX_LEN - len(sample)
        if err == 0:
            pad_samples.append(sample)
        else:
            pad_data = [0 for i in range(0, err)]
            pad_samples.append(pad_data + sample)
    print("\nFinal Samples")
    print(np.array(pad_samples))
    
    return pad_samples
    
test_samples = get_samples(test_merge_datas)
test_samples = get_padding_samples(test_samples)

training_samples = get_samples(merge_datas)
training_samples = get_padding_samples(training_samples)

Samples Before Padding Process
[list([1, 1, 6, 13, 53, 0.056]) list([1, 1, 6, 13, 53, 0.056, 0.053])
 list([1, 1, 6, 13, 53, 0.056, 0.053, 0.062])
 list([1, 1, 6, 13, 53, 0.056, 0.053, 0.062, 0.063])
 list([1, 1, 6, 13, 53, 0.056, 0.053, 0.062, 0.063, 0.061])
 list([1, 1, 6, 13, 53, 0.056, 0.053, 0.062, 0.063, 0.061, 0.087])
 list([1, 1, 6, 13, 53, 0.056, 0.053, 0.062, 0.063, 0.061, 0.087, 0.05])
 list([1, 1, 6, 13, 53, 0.056, 0.053, 0.062, 0.063, 0.061, 0.087, 0.05, 0.046])
 list([1, 1, 6, 13, 53, 0.056, 0.053, 0.062, 0.063, 0.061, 0.087, 0.05, 0.046, 0.051])
 list([1, 1, 6, 13, 53, 0.056, 0.053, 0.062, 0.063, 0.061, 0.087, 0.05, 0.046, 0.051, 0.06])
 list([1, 1, 6, 13, 53, 0.056, 0.053, 0.062, 0.063, 0.061, 0.087, 0.05, 0.046, 0.051, 0.06, 0.06])
 list([1, 1, 6, 13, 53, 0.056, 0.053, 0.062, 0.063, 0.061, 0.087, 0.05, 0.046, 0.051, 0.06, 0.06, 0.056])
 list([1, 1, 6, 13, 53, 0.056, 0.053, 0.062, 0.063, 0.061, 0.087, 0.05, 0.046, 0.051, 0.06, 0.06, 0.056, 0.042])
 list([1, 1, 6, 13, 53

<ipython-input-612-f7267c0eac4d>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  print(np.array(samples))


In [613]:
# Set Tranining Data
training_samples = np.array(training_samples)

train_X = training_samples[:,:-1]
train_y = training_samples[:,-1]

# Set Test Data
test_samples = np.array(test_samples)
test_X = test_samples[:,:-1]
test_y = test_samples[:,-1]

print("Input Data For Training")
print(np.array(train_X))
print()
print("Output Data For Training")
print(np.array(train_y))

print("Input Data For Test")
print(np.array(test_X))
print()
print("Output Data For Test")
print(np.array(test_y))

Input Data For Training
[[0.00e+00 0.00e+00 0.00e+00 ... 2.00e+00 2.00e+01 7.30e+01]
 [0.00e+00 0.00e+00 0.00e+00 ... 2.00e+01 7.30e+01 3.41e-01]
 [0.00e+00 0.00e+00 0.00e+00 ... 7.30e+01 3.41e-01 3.37e-01]
 ...
 [0.00e+00 0.00e+00 1.00e+00 ... 3.80e-02 6.10e-02 5.90e-02]
 [0.00e+00 1.00e+00 2.00e+00 ... 6.10e-02 5.90e-02 3.00e-02]
 [1.00e+00 2.00e+00 6.00e+00 ... 5.90e-02 3.00e-02 5.60e-02]]

Output Data For Training
[0.341 0.337 0.324 ... 0.03  0.056 0.058]
Input Data For Test
[[ 0.     0.     0.    ...  6.    13.    53.   ]
 [ 0.     0.     0.    ... 13.    53.     0.056]
 [ 0.     0.     0.    ... 53.     0.056  0.053]
 ...
 [ 0.     0.     1.    ...  0.11   0.274  0.28 ]
 [ 0.     1.     2.    ...  0.274  0.28   0.279]
 [ 1.     2.     2.    ...  0.28   0.279  0.292]]

Output Data For Test
[0.056 0.053 0.062 0.063 0.061 0.087 0.05  0.046 0.051 0.06  0.06  0.056
 0.042 0.049 0.06  0.06  0.046 0.043 0.125 0.197 0.208 0.239 0.248 0.169
 0.275 0.201 0.176 0.21  0.199 0.202 0.199 0.252

In [331]:
model = Sequential()
# model.add(Embedding(value_size, input_length=28))
model.add(LSTM(128, input_shape=(28, 1),return_sequences=True, activation='softmax'))
model.add(Dense(1,activation='softmax'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [332]:
print(len(train_X), train_X.shape, train_X)
train_X = train_X.reshape(train_X.shape[0], train_X.shape[1], 1)
print(len(train_X), train_X.shape, train_X)
print(len(train_y), train_y.shape, train_y)

1872 (1872, 28) [[  0   0   0 ...   2  20  73]
 [  0   0   0 ...  20  73 342]
 [  0   0   0 ...  73 342 338]
 ...
 [  0   0   1 ...  39  62  60]
 [  0   1   2 ...  62  60  31]
 [  1   2   6 ...  60  31  57]]
1872 (1872, 28, 1) [[[  0]
  [  0]
  [  0]
  ...
  [  2]
  [ 20]
  [ 73]]

 [[  0]
  [  0]
  [  0]
  ...
  [ 20]
  [ 73]
  [342]]

 [[  0]
  [  0]
  [  0]
  ...
  [ 73]
  [342]
  [338]]

 ...

 [[  0]
  [  0]
  [  1]
  ...
  [ 39]
  [ 62]
  [ 60]]

 [[  0]
  [  1]
  [  2]
  ...
  [ 62]
  [ 60]
  [ 31]]

 [[  1]
  [  2]
  [  6]
  ...
  [ 60]
  [ 31]
  [ 57]]]
1872 (1872,) [342 338 325 ...  31  57  59]


In [333]:
model.fit(train_X, train_y, epochs=200, batch_size=1 ,verbose=2)

Epoch 1/200
1872/1872 - 7s - loss: 32817.1172 - accuracy: 0.0000e+00
Epoch 2/200
1872/1872 - 6s - loss: 32817.1211 - accuracy: 0.0000e+00
Epoch 3/200
1872/1872 - 6s - loss: 32817.1211 - accuracy: 0.0000e+00
Epoch 4/200
1872/1872 - 6s - loss: 32817.1211 - accuracy: 0.0000e+00
Epoch 5/200


KeyboardInterrupt: 

In [318]:
test = test_X[2]
test = test.reshape(28,1,1)
print(test)
result = model.predict(test, verbose=0)
print(result)

[[[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  0]]

 [[  1]]

 [[  3]]

 [[  2]]

 [[  8]]

 [[ 55]]

 [[162]]

 [[160]]]
[[[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]

 [[1.]]]


In [385]:
print(train_X)
print(train_y)

one_hot_y = to_categorical(train_y,num_classes= value_size)
print(one_hot_y)

[[  0   0   0 ...   3  11  62]
 [  0   0   0 ...  11  62 184]
 [  0   0   0 ...  62 184 236]
 ...
 [  0   0   1 ...  39  62  60]
 [  0   1   2 ...  62  60  31]
 [  1   2   6 ...  60  31  57]]
[184 236 168 ...  31  57  59]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [614]:
model_2 = Sequential()
model_2.add(Embedding(value_size, 10, input_length=28))
model_2.add(LSTM(64))
model_2.add(Dense(value_size, activation='softmax'))
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_2.summary()

Model: "sequential_51"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 28, 10)            18720     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                19200     
_________________________________________________________________
dense_41 (Dense)             (None, 1872)              121680    
Total params: 159,600
Trainable params: 159,600
Non-trainable params: 0
_________________________________________________________________


In [615]:
model_2.fit(train_X, one_hot_y, epochs=1000, verbose=2)

Epoch 1/1000
59/59 - 2s - loss: 6.7429 - accuracy: 0.0208
Epoch 2/1000
59/59 - 0s - loss: 5.5609 - accuracy: 0.0321
Epoch 3/1000
59/59 - 0s - loss: 5.4822 - accuracy: 0.0321
Epoch 4/1000
59/59 - 0s - loss: 5.4591 - accuracy: 0.0321
Epoch 5/1000
59/59 - 0s - loss: 5.4474 - accuracy: 0.0321
Epoch 6/1000
59/59 - 0s - loss: 5.4450 - accuracy: 0.0321
Epoch 7/1000
59/59 - 0s - loss: 5.4399 - accuracy: 0.0321
Epoch 8/1000
59/59 - 0s - loss: 5.4384 - accuracy: 0.0321
Epoch 9/1000
59/59 - 0s - loss: 5.4307 - accuracy: 0.0321
Epoch 10/1000
59/59 - 0s - loss: 5.4292 - accuracy: 0.0321
Epoch 11/1000
59/59 - 0s - loss: 5.4280 - accuracy: 0.0321
Epoch 12/1000
59/59 - 0s - loss: 5.4299 - accuracy: 0.0321
Epoch 13/1000
59/59 - 0s - loss: 5.4252 - accuracy: 0.0321
Epoch 14/1000
59/59 - 0s - loss: 5.4238 - accuracy: 0.0321
Epoch 15/1000
59/59 - 0s - loss: 5.4226 - accuracy: 0.0321
Epoch 16/1000
59/59 - 0s - loss: 5.4220 - accuracy: 0.0321
Epoch 17/1000
59/59 - 0s - loss: 5.4173 - accuracy: 0.0321
Epoch 

Epoch 140/1000
59/59 - 0s - loss: 4.1138 - accuracy: 0.0839
Epoch 141/1000
59/59 - 0s - loss: 4.1154 - accuracy: 0.0791
Epoch 142/1000
59/59 - 0s - loss: 4.1307 - accuracy: 0.0849
Epoch 143/1000
59/59 - 0s - loss: 4.0785 - accuracy: 0.0892
Epoch 144/1000
59/59 - 0s - loss: 4.0777 - accuracy: 0.0946
Epoch 145/1000
59/59 - 0s - loss: 4.0626 - accuracy: 0.0924
Epoch 146/1000
59/59 - 0s - loss: 4.0645 - accuracy: 0.0908
Epoch 147/1000
59/59 - 0s - loss: 4.0575 - accuracy: 0.0876
Epoch 148/1000
59/59 - 0s - loss: 4.0324 - accuracy: 0.0924
Epoch 149/1000
59/59 - 0s - loss: 4.0760 - accuracy: 0.0972
Epoch 150/1000
59/59 - 0s - loss: 4.0347 - accuracy: 0.0962
Epoch 151/1000
59/59 - 0s - loss: 4.0086 - accuracy: 0.0919
Epoch 152/1000
59/59 - 0s - loss: 4.0150 - accuracy: 0.0967
Epoch 153/1000
59/59 - 0s - loss: 3.9930 - accuracy: 0.0951
Epoch 154/1000
59/59 - 0s - loss: 3.9891 - accuracy: 0.0972
Epoch 155/1000
59/59 - 0s - loss: 3.9667 - accuracy: 0.0967
Epoch 156/1000
59/59 - 0s - loss: 3.9616

Epoch 277/1000
59/59 - 0s - loss: 3.0625 - accuracy: 0.2286
Epoch 278/1000
59/59 - 0s - loss: 3.0585 - accuracy: 0.2265
Epoch 279/1000
59/59 - 0s - loss: 3.0293 - accuracy: 0.2409
Epoch 280/1000
59/59 - 0s - loss: 3.0239 - accuracy: 0.2420
Epoch 281/1000
59/59 - 0s - loss: 3.0092 - accuracy: 0.2457
Epoch 282/1000
59/59 - 0s - loss: 3.0156 - accuracy: 0.2457
Epoch 283/1000
59/59 - 0s - loss: 3.0146 - accuracy: 0.2329
Epoch 284/1000
59/59 - 0s - loss: 3.0055 - accuracy: 0.2415
Epoch 285/1000
59/59 - 0s - loss: 3.0342 - accuracy: 0.2468
Epoch 286/1000
59/59 - 0s - loss: 3.0300 - accuracy: 0.2399
Epoch 287/1000
59/59 - 0s - loss: 3.0216 - accuracy: 0.2356
Epoch 288/1000
59/59 - 0s - loss: 2.9996 - accuracy: 0.2479
Epoch 289/1000
59/59 - 0s - loss: 2.9773 - accuracy: 0.2543
Epoch 290/1000
59/59 - 0s - loss: 2.9988 - accuracy: 0.2463
Epoch 291/1000
59/59 - 0s - loss: 2.9758 - accuracy: 0.2532
Epoch 292/1000
59/59 - 0s - loss: 2.9495 - accuracy: 0.2516
Epoch 293/1000
59/59 - 0s - loss: 2.9920

Epoch 414/1000
59/59 - 0s - loss: 2.3127 - accuracy: 0.4038
Epoch 415/1000
59/59 - 0s - loss: 2.3018 - accuracy: 0.3990
Epoch 416/1000
59/59 - 0s - loss: 2.3340 - accuracy: 0.3958
Epoch 417/1000
59/59 - 0s - loss: 2.3258 - accuracy: 0.3921
Epoch 418/1000
59/59 - 0s - loss: 2.3394 - accuracy: 0.3942
Epoch 419/1000
59/59 - 0s - loss: 2.3091 - accuracy: 0.4006
Epoch 420/1000
59/59 - 0s - loss: 2.2881 - accuracy: 0.4092
Epoch 421/1000
59/59 - 0s - loss: 2.2812 - accuracy: 0.4033
Epoch 422/1000
59/59 - 0s - loss: 2.2715 - accuracy: 0.4172
Epoch 423/1000
59/59 - 0s - loss: 2.3561 - accuracy: 0.3921
Epoch 424/1000
59/59 - 0s - loss: 2.2949 - accuracy: 0.4060
Epoch 425/1000
59/59 - 0s - loss: 2.3944 - accuracy: 0.3803
Epoch 426/1000
59/59 - 0s - loss: 2.3628 - accuracy: 0.3819
Epoch 427/1000
59/59 - 0s - loss: 2.2818 - accuracy: 0.3964
Epoch 428/1000
59/59 - 0s - loss: 2.2823 - accuracy: 0.4028
Epoch 429/1000
59/59 - 0s - loss: 2.2619 - accuracy: 0.4092
Epoch 430/1000
59/59 - 0s - loss: 2.2505

Epoch 551/1000
59/59 - 0s - loss: 1.9291 - accuracy: 0.4781
Epoch 552/1000
59/59 - 0s - loss: 1.9928 - accuracy: 0.4658
Epoch 553/1000
59/59 - 0s - loss: 1.8491 - accuracy: 0.5150
Epoch 554/1000
59/59 - 0s - loss: 1.8685 - accuracy: 0.5080
Epoch 555/1000
59/59 - 0s - loss: 1.8076 - accuracy: 0.5326
Epoch 556/1000
59/59 - 0s - loss: 1.8913 - accuracy: 0.5091
Epoch 557/1000
59/59 - 0s - loss: 2.0635 - accuracy: 0.4578
Epoch 558/1000
59/59 - 0s - loss: 2.5050 - accuracy: 0.4300
Epoch 559/1000
59/59 - 0s - loss: 2.4028 - accuracy: 0.4284
Epoch 560/1000
59/59 - 0s - loss: 2.3111 - accuracy: 0.3985
Epoch 561/1000
59/59 - 0s - loss: 2.1674 - accuracy: 0.4359
Epoch 562/1000
59/59 - 0s - loss: 1.9394 - accuracy: 0.4936
Epoch 563/1000
59/59 - 0s - loss: 1.8608 - accuracy: 0.5107
Epoch 564/1000
59/59 - 0s - loss: 1.8301 - accuracy: 0.5326
Epoch 565/1000
59/59 - 0s - loss: 1.8113 - accuracy: 0.5342
Epoch 566/1000
59/59 - 0s - loss: 1.7808 - accuracy: 0.5331
Epoch 567/1000
59/59 - 0s - loss: 1.7660

Epoch 688/1000
59/59 - 0s - loss: 1.4897 - accuracy: 0.6042
Epoch 689/1000
59/59 - 0s - loss: 1.4474 - accuracy: 0.6277
Epoch 690/1000
59/59 - 0s - loss: 1.4347 - accuracy: 0.6330
Epoch 691/1000
59/59 - 0s - loss: 1.3948 - accuracy: 0.6394
Epoch 692/1000
59/59 - 0s - loss: 1.4002 - accuracy: 0.6384
Epoch 693/1000
59/59 - 0s - loss: 1.3870 - accuracy: 0.6426
Epoch 694/1000
59/59 - 0s - loss: 1.3831 - accuracy: 0.6432
Epoch 695/1000
59/59 - 0s - loss: 1.4138 - accuracy: 0.6287
Epoch 696/1000
59/59 - 0s - loss: 1.3911 - accuracy: 0.6426
Epoch 697/1000
59/59 - 0s - loss: 1.5181 - accuracy: 0.5967
Epoch 698/1000
59/59 - 0s - loss: 1.5201 - accuracy: 0.6052
Epoch 699/1000
59/59 - 0s - loss: 1.5005 - accuracy: 0.6010
Epoch 700/1000
59/59 - 0s - loss: 1.5566 - accuracy: 0.5785
Epoch 701/1000
59/59 - 0s - loss: 1.4540 - accuracy: 0.6213
Epoch 702/1000
59/59 - 0s - loss: 1.5048 - accuracy: 0.5988
Epoch 703/1000
59/59 - 0s - loss: 1.5067 - accuracy: 0.5929
Epoch 704/1000
59/59 - 0s - loss: 1.4464

Epoch 825/1000
59/59 - 0s - loss: 1.1391 - accuracy: 0.7105
Epoch 826/1000
59/59 - 0s - loss: 1.1407 - accuracy: 0.7190
Epoch 827/1000
59/59 - 0s - loss: 1.1371 - accuracy: 0.7062
Epoch 828/1000
59/59 - 0s - loss: 1.1643 - accuracy: 0.7035
Epoch 829/1000
59/59 - 0s - loss: 1.2625 - accuracy: 0.6651
Epoch 830/1000
59/59 - 0s - loss: 1.1821 - accuracy: 0.6998
Epoch 831/1000
59/59 - 0s - loss: 1.2303 - accuracy: 0.6768
Epoch 832/1000
59/59 - 0s - loss: 1.1680 - accuracy: 0.6976
Epoch 833/1000
59/59 - 0s - loss: 1.2444 - accuracy: 0.6800
Epoch 834/1000
59/59 - 0s - loss: 1.3400 - accuracy: 0.6474
Epoch 835/1000
59/59 - 0s - loss: 1.4562 - accuracy: 0.6127
Epoch 836/1000
59/59 - 0s - loss: 1.5351 - accuracy: 0.5753
Epoch 837/1000
59/59 - 0s - loss: 1.2350 - accuracy: 0.6870
Epoch 838/1000
59/59 - 0s - loss: 1.1600 - accuracy: 0.6987
Epoch 839/1000
59/59 - 0s - loss: 1.1537 - accuracy: 0.7014
Epoch 840/1000
59/59 - 0s - loss: 1.1767 - accuracy: 0.6960
Epoch 841/1000
59/59 - 0s - loss: 1.1351

Epoch 962/1000
59/59 - 0s - loss: 0.9101 - accuracy: 0.7644
Epoch 963/1000
59/59 - 0s - loss: 0.9535 - accuracy: 0.7505
Epoch 964/1000
59/59 - 0s - loss: 0.9532 - accuracy: 0.7543
Epoch 965/1000
59/59 - 0s - loss: 0.9772 - accuracy: 0.7548
Epoch 966/1000
59/59 - 0s - loss: 1.2002 - accuracy: 0.6768
Epoch 967/1000
59/59 - 0s - loss: 1.1967 - accuracy: 0.6715
Epoch 968/1000
59/59 - 0s - loss: 1.0400 - accuracy: 0.7244
Epoch 969/1000
59/59 - 0s - loss: 1.0039 - accuracy: 0.7404
Epoch 970/1000
59/59 - 0s - loss: 0.9747 - accuracy: 0.7399
Epoch 971/1000
59/59 - 0s - loss: 0.9699 - accuracy: 0.7425
Epoch 972/1000
59/59 - 0s - loss: 0.9481 - accuracy: 0.7516
Epoch 973/1000
59/59 - 0s - loss: 0.9395 - accuracy: 0.7580
Epoch 974/1000
59/59 - 0s - loss: 1.0278 - accuracy: 0.7254
Epoch 975/1000
59/59 - 0s - loss: 0.9848 - accuracy: 0.7377
Epoch 976/1000
59/59 - 0s - loss: 0.9323 - accuracy: 0.7612
Epoch 977/1000
59/59 - 0s - loss: 1.0453 - accuracy: 0.7254
Epoch 978/1000
59/59 - 0s - loss: 0.9482

In [550]:
umm = 38
result = model_2.predict_classes([test_X[umm].tolist()])
print("result",y[result[0]])
print("real value", test_y[umm])

result 0.263
real value 0.06


In [617]:
# visual
for i in range(0, round(len(test_X) / 24)):
    start_idx = 24 * i
    end_idx = (24 * (i + 1))
    
    real_pattern = test_X[end_idx-1][-23:]
    real_pattern = np.append(real_pattern, [test_y[end_idx-1]])
    
    print("real_pattern:",real_pattern)
    
    predict_pattern = [test_X[start_idx+1][-1].tolist()]
    for p in range(start_idx + 1, end_idx):
        result = model_2.predict_classes([test_X[p].tolist()])
        predict_pattern.append(y[result[0]])
    print("predict_pattern:", predict_pattern)

real_pattern: [0.056 0.053 0.062 0.063 0.061 0.087 0.05  0.046 0.051 0.06  0.06  0.056
 0.042 0.049 0.06  0.06  0.046 0.043 0.125 0.197 0.208 0.239 0.248 0.169]
predict_pattern: [0.056, 0.299, 0.263, 0.344, 0.175, 0.175, 0.175, 0.175, 0.208, 0.208, 0.208, 0.208, 0.208, 0.208, 0.208, 0.411, 0.072, 0.292, 0.319, 0.247, 0.334, 0.201, 0.41, 0.184]
real_pattern: [0.275 0.201 0.176 0.21  0.199 0.202 0.199 0.252 0.213 0.173 0.186 0.188
 0.171 0.191 0.175 0.168 0.196 0.177 0.218 0.2   0.206 0.202 0.201 0.196]
predict_pattern: [0.275, 0.173, 0.173, 0.173, 0.173, 0.173, 0.173, 0.32, 0.32, 0.2, 0.278, 0.278, 0.098, 0.098, 0.098, 0.098, 0.057, 0.057, 0.243, 0.173, 0.242, 0.213, 0.208, 0.324]
real_pattern: [0.176 0.175 0.211 0.211 0.204 0.202 0.176 0.241 0.175 0.174 0.178 0.184
 0.185 0.175 0.177 0.175 0.175 0.176 0.215 0.201 0.206 0.2   0.197 0.197]
predict_pattern: [0.176, 0.282, 0.292, 0.21, 0.345, 0.29, 0.257, 0.233, 0.056, 0.243, 0.176, 0.208, 0.043, 0.056, 0.214, 0.217, 0.31, 0.31, 0.18, 0.04

In [ ]:
client.close()

In [574]:
test_merge_datas

,2018-05-01,2019-04-17,2019-04-06,2018-05-20,2019-03-15,2019-04-28,2019-03-23,2019-04-08,2018-05-26,2019-03-09,2019-04-02,2018-05-23,2018-05-24,2018-05-02
0,0.341,0.073,0.065,0.208,0.064,0.290,0.035,0.091,0.310,0.174,0.048,0.257,0.176,0.275
1,0.337,0.061,0.043,0.208,0.039,0.267,0.064,0.088,0.255,0.089,0.075,0.181,0.175,0.201
2,0.324,0.037,0.061,0.208,0.052,0.244,0.063,0.107,0.211,0.083,0.078,0.174,0.211,0.176
3,0.319,0.054,0.060,0.208,0.061,0.276,0.040,0.073,0.207,0.071,0.049,0.171,0.211,0.210
4,0.235,0.062,0.028,0.208,0.053,0.232,0.049,0.094,0.202,0.097,0.066,0.174,0.204,0.199
5,0.169,0.050,0.060,0.208,0.037,0.135,0.061,0.100,0.202,0.101,0.073,0.171,0.202,0.202
6,0.200,0.040,0.058,0.208,0.102,0.093,0.051,0.072,0.172,0.076,0.056,0.172,0.176,0.199
7,0.171,0.110,0.035,0.208,0.057,0.093,0.039,0.090,0.193,0.112,0.047,0.217,0.241,0.252
8,0.170,0.059,0.053,0.208,0.038,0.066,0.059,0.097,0.181,0.260,0.057,0.188,0.175,0.213
9,0.172,0.032,0.059,0.208,0.046,0.089,0.059,0.073,0.176,0.346,0.048,0.189,0.174,0.173
